In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import yaml
import subprocess

from glob import glob
from utils import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
paths = ["../data/bio/bio-nr-1010-size-108.graph", "../data/bio/bio-nr-1015-size-42.graph",
 "../data/bio/bio-nr-1020-size-44.graph", "../data/bio/bio-nr-1023-size-49.graph",
 "../data/bio/bio-nr-1024-size-13.graph", "../data/bio/bio-nr-1029-size-37.graph",
 "../data/bio/bio-nr-1043-size-39.graph", "../data/bio/bio-nr-105-size-20.graph",
 "../data/bio/bio-nr-1041-size-13.graph", "../data/bio/bio-nr-1061-size-9.graph",
 "../data/bio/bio-nr-1064-size-34.graph", "../data/bio/bio-nr-1084-size-26.graph",
 "../data/bio/bio-nr-1085-size-21.graph", "../data/bio/bio-nr-1092-size-30.graph",
 "../data/bio/bio-nr-1094-size-33.graph", "../data/bio/bio-nr-1099-size-29.graph",
 "../data/bio/bio-nr-11-size-22.graph", "../data/bio/bio-nr-1105-size-36.graph",
 "../data/bio/bio-nr-1109-size-37.graph", "../data/bio/bio-nr-1141-size-48.graph",
 "../data/bio/bio-nr-1142-size-29.graph", "../data/bio/bio-nr-1160-size-25.graph",
 "../data/bio/bio-nr-1187-size-30.graph", "../data/bio/bio-nr-1210-size-28.graph",
 "../data/bio/bio-nr-122-size-39.graph", "../data/bio/bio-nr-1235-size-44.graph",
 "../data/bio/bio-nr-1236-size-47.graph", "../data/bio/bio-nr-1258-size-35.graph",
 "../data/bio/bio-nr-1263-size-42.graph", "../data/bio/bio-nr-1272-size-31.graph",
 "../data/bio/bio-nr-1310-size-104.graph", "../data/bio/bio-nr-1313-size-31.graph",
 "../data/bio/bio-nr-1330-size-27.graph", "../data/bio/bio-nr-1355-size-39.graph",
 "../data/bio/bio-nr-1356-size-45.graph", "../data/bio/bio-nr-1382-size-20.graph",
 "../data/bio/bio-nr-1390-size-33.graph", "../data/bio/bio-nr-1407-size-39.graph",
 "../data/bio/bio-nr-1410-size-68.graph", "../data/bio/bio-nr-1413-size-20.graph",
 "../data/bio/bio-nr-1418-size-29.graph", "../data/bio/bio-nr-1420-size-40.graph",
 "../data/bio/bio-nr-1435-size-18.graph", "../data/bio/bio-nr-1466-size-40.graph",
 "../data/bio/bio-nr-1467-size-65.graph", "../data/bio/bio-nr-1487-size-23.graph",
 "../data/bio/bio-nr-1501-size-31.graph", "../data/bio/bio-nr-1505-size-26.graph",
 "../data/bio/bio-nr-1510-size-39.graph", "../data/bio/bio-nr-1516-size-69.graph",
 "../data/bio/bio-nr-1529-size-39.graph", "../data/bio/bio-nr-1555-size-24.graph",
 "../data/bio/bio-nr-1565-size-29.graph", "../data/bio/bio-nr-1569-size-44.graph",
 "../data/bio/bio-nr-1581-size-20.graph", "../data/bio/bio-nr-1592-size-16.graph",
 "../data/bio/bio-nr-1634-size-39.graph", "../data/bio/bio-nr-1657-size-15.graph",
 "../data/bio/bio-nr-1663-size-24.graph", "../data/bio/bio-nr-1671-size-25.graph",
 "../data/bio/bio-nr-1673-size-33.graph", "../data/bio/bio-nr-1681-size-19.graph",
 "../data/bio/bio-nr-1685-size-17.graph", "../data/bio/bio-nr-1688-size-28.graph",
 "../data/bio/bio-nr-1693-size-29.graph", "../data/bio/bio-nr-170-size-57.graph",
 "../data/bio/bio-nr-171-size-89.graph", "../data/bio/bio-nr-1732-size-21.graph",
 "../data/bio/bio-nr-1744-size-34.graph", "../data/bio/bio-nr-1752-size-19.graph",
 "../data/bio/bio-nr-1755-size-38.graph"
]
#paths = ["../data/bio/bio-nr-1541-size-12.graph"]
#paths = glob("../data/bio/*.graph")
#paths = glob("../data/duplication-divergence/*.graph")
#paths = glob("../data/powerlaw-cluster/*.graph")
#paths = glob("../data/barabasi-albert/*.graph")
#paths = ["../data/misc/lesmis.graph", "../data/misc/karate.graph", "../data/misc/grass_web.graph"]
#paths = glob("../data/bio-subset-A/*.graph")

In [ ]:
F = "C5P5"
for path in paths:
    print(path)
    try:
        out = subprocess.run(["../cmake-build-release/ilp", "--input", f"{path}", "--F", F], timeout=100, capture_output=True)
    except subprocess.TimeoutExpired:
        print("timeout")
        continue

    doc = yaml.safe_load("\n".join(out.stdout.decode().split("\n")[1:]))
    if len(doc['solutions']) == 0:
        continue
    solution = doc['solutions'][0]
    edits = list(map(tuple, solution['edits']))
    
    
    G, S = read_metis_graph(path)
    costs = np.abs(S)
    
    # for visualization
    # costs = 0.5 + 1.5 * costs
    costs = np.log1p(costs)
    
    costs = 0.5 + 2 * costs / np.array([costs[u, v] for (u, v) in G.edges()]).mean()

    
    unedited = [e for e in G.edges() if e not in edits]
    deleted = [e for e in G.edges() if e in edits]
    inserted = [e for e in edits if e not in G.edges()]
    
    G_edited = G.copy()
    G_edited.remove_edges_from(deleted)
    G_edited.add_edges_from(inserted)
    
    c = np.zeros(G_edited.number_of_nodes())
    for i, S in enumerate(nx.connected_components(G_edited)):
        for u in S:
            c[u] = i

    
            
    # cmap = "tab10"
    cmap = "Paired"
    edge_cmap = plt.get_cmap("Greys")

    
    fig, ax1 = plt.subplots(figsize=(8, 8))
    ax1.set_aspect("equal")
    ax1.axis("off")
    
    pos = nx.nx_pydot.graphviz_layout(G, 'neato')
    nx.draw_networkx_nodes(G, pos, ax=ax1, node_size=80, node_color=c, cmap=cmap)
    for (edges, color) in [(unedited, [costs[uv] for uv in unedited]), (deleted, 'r'), (inserted, 'g')]:
        nx.draw_networkx_edges(G, pos, ax=ax1,
                               edgelist=edges,
                               width=[costs[u,v] for (u, v) in edges], edge_color=color,
                               edge_cmap=edge_cmap, edge_vmin=0, edge_vmax=costs.max())

    # fig.tight_layout()
    plt.savefig(f"../scripts/figures/edits/{F}-edit-{G.name}.pdf", bbox_inches="tight")
    plt.show()
    
    
    
    fig, ax2 = plt.subplots(figsize=(8, 8))
    ax2.set_aspect("equal")
    ax2.axis("off")
    
    pos = nx.nx_pydot.graphviz_layout(G_edited, 'neato')
    nx.draw_networkx_nodes(G_edited, pos, ax=ax2, node_size=80, node_color=c, cmap=cmap)
    nx.draw_networkx_edges(G_edited, pos, ax=ax2,
                           edgelist=G_edited.edges(),
                           width=[costs[u,v] for (u, v) in G_edited.edges()],
                           edge_color=[costs[uv] for uv in G_edited.edges()],
                           edge_cmap=edge_cmap, edge_vmin=0, edge_vmax=costs.max())
    
    # fig.tight_layout()
    plt.savefig(f"../scripts/figures/edits/{F}-edit-{G.name}-edited.pdf", bbox_inches="tight")
    plt.show()
    
    

    num_connected_components = len(list(nx.connected_components(G_edited)))
    info = dict(
        num_edits=len(edits),
        num_connected_components=(1, num_connected_components),
        diameter=([nx.diameter(G.subgraph(S)) for S in nx.connected_components(G)], [nx.diameter(G_edited.subgraph(S)) for S in nx.connected_components(G_edited)]),
        average_clustering=(nx.average_clustering(G), nx.average_clustering(G_edited))
    )
    print(info)
    print(info['diameter'][0][0] - max(info['diameter'][1]))
    print(info['average_clustering'][0] - info['average_clustering'][1])